# Step 2:
## Extracting SNPs (Variants) from PGS Catalog Score Files
### By: Saniya Khullar :)

Please verify that all of the score files have the same reference build (e.g. Hg19/GRCh37 or Hg38/GRCh38)
This code is for hg38 (human reference genome: GRCh38).

In [1]:
import os
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from collections import defaultdict
import math


/Users/sk37/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
parent_file = "/Users/sk37/Desktop/hg38_demo/"
dna_nexus_project_folder = "IBD_Projects_Saniya:/hg38_saniya_demo/" # the main folder we are working with in dnanexus
hg38_genotype_imputation_type = "topmed"
snp_threshold = 8000  # max SNPs per file
scores_fp = parent_file + "score_file/"

In [3]:
scores_list = os.listdir(scores_fp)
score_files_list = []
for score_file in scores_list:
    if ".txt" in score_file:
        score_files_list.append(score_file)
score_files_list

['PGS005072.txt']

In [5]:
# len(os.listdir("/Users/sk37/Desktop/newest_demo/score_file/per_chrom_snp_files_GRCh37_combined_PGS005072/ranges_group/"))

In [6]:
for score_name in score_files_list:
    input_file = scores_fp + score_name
    dir_name = os.path.dirname(os.path.abspath(input_file))
    print(dir_name)

    # Read file
    with open(input_file, "r") as f:
        lines = f.readlines()

    # Extract pgs_id and genome_build from header
    pgs_id = None
    genome_build = None
    for line in lines:
        if line.startswith("#pgs_id="):
            pgs_id = line.strip().split("=")[1]
        elif line.startswith("#genome_build="):
            genome_build = line.strip().split("=")[1]

    if not pgs_id or not genome_build:
        raise ValueError("pgs_id or genome_build missing in file header.")
    print(f":) Please note: pgs_id = {pgs_id} and genome_build = {genome_build}")

    # Extract SNP table (start at header "rsID")
    snp_lines = []
    header_found = False
    for line in tqdm(lines, desc = ":) going through each line to find start of score data"):
        if line.startswith("#") == False:
        # if line.startswith("rsID"):
            header_found = True
            snp_lines.append(line.strip())  # include header
            continue
        if header_found:
            if line.strip() and not line.startswith("#"):
                snp_lines.append(line.strip())

    # Save SNP table to TSV
    output_tsv = os.path.join(dir_name, f"{pgs_id}_SNPs_{genome_build}.tsv")
    with open(output_tsv, "w") as f:
        f.write("\n".join(snp_lines) + "\n")

    print(f"Saved SNP table to {output_tsv}")

/Users/sk37/Desktop/hg38_demo/score_file
:) Please note: pgs_id = PGS005072 and genome_build = GRCh38


:) going through each line to find start of score data: 100%|██████████| 1273906/1273906 [00:00<00:00, 3305461.15it/s]


Saved SNP table to /Users/sk37/Desktop/hg38_demo/score_file/PGS005072_SNPs_GRCh38.tsv


In [7]:
scores_list = os.listdir(scores_fp)
score_files_list = []
for score_file in scores_list:
    if ".tsv" in score_file:
        score_files_list.append(score_file)
score_files_list

['PGS005072_SNPs_GRCh38.tsv']

In [8]:
df_list = []
for score_name in tqdm(score_files_list):
    input_file = scores_fp + score_name
    df = pd.read_csv(input_file, sep = "\t")
    df_list.append(df)
combined = pd.concat(df_list, axis=0, join='outer', ignore_index=True)
combined


100%|██████████| 1/1 [00:00<00:00,  1.82it/s]


,rsID,chr_name,chr_position,effect_allele,other_allele,effect_weight
0,rs3131972,1,817341,G,A,0.000021
1,rs3131969,1,818802,G,A,0.000028
2,rs1048488,1,825532,T,C,0.000008
3,rs12562034,1,833068,A,G,0.000004
4,rs4040617,1,843942,G,A,-0.000014
...,...,...,...,...,...,...
1273886,rs2240171,9,133361429,G,A,-0.000064
1273887,rs11103132,9,135684056,C,A,-0.000044
1273888,rs17040507,9,135820929,A,G,0.000022
1273889,rs2275159,9,136726874,A,G,0.000023


In [9]:
scores_list_names = '_'.join([i.split("_")[0] for i in score_files_list])
scores_list_names

'PGS005072'

In [10]:
total_unique_variants_df = combined[["chr_name", "chr_position"]].drop_duplicates()
print(f":) Please note that there are {total_unique_variants_df.shape[0]:,} unique {genome_build} variants combined across these {len(score_files_list)} score files: {scores_list_names}")
total_unique_variants_df

:) Please note that there are 1,273,891 unique GRCh38 variants combined across these 1 score files: PGS005072


,chr_name,chr_position
0,1,817341
1,1,818802
2,1,825532
3,1,833068
4,1,843942
...,...,...
1273886,9,133361429
1273887,9,135684056
1273888,9,135820929
1273889,9,136726874


In [11]:
combined

,rsID,chr_name,chr_position,effect_allele,other_allele,effect_weight
0,rs3131972,1,817341,G,A,0.000021
1,rs3131969,1,818802,G,A,0.000028
2,rs1048488,1,825532,T,C,0.000008
3,rs12562034,1,833068,A,G,0.000004
4,rs4040617,1,843942,G,A,-0.000014
...,...,...,...,...,...,...
1273886,rs2240171,9,133361429,G,A,-0.000064
1273887,rs11103132,9,135684056,C,A,-0.000044
1273888,rs17040507,9,135820929,A,G,0.000022
1273889,rs2275159,9,136726874,A,G,0.000023


In [12]:
# Create subdirectory for per-chromosome files
subdir = os.path.join(dir_name, f"per_chrom_snp_files_{genome_build}_combined_{scores_list_names}")
subdir1 = os.path.join(subdir, "ranges_group")

os.makedirs(subdir, exist_ok=True)
os.makedirs(subdir1, exist_ok=True)

# Ordered dicts for uniqueness while preserving input order
chrom_ranges = defaultdict(dict)

chrom_found_list = []

# Collect SNPs
# Parse header dynamically
header = combined.columns.tolist()
chr_name_idx = header.index("chr_name")
chr_pos_idx = header.index("chr_position")

snp_lines_to_use = combined.values.tolist()
# Collect SNPs
for row in tqdm(snp_lines_to_use):  # skip header
    cols = [str(i) for i in row]#.strip().split("\t")
    chr_name = cols[chr_name_idx]
    chr_pos = int(cols[chr_pos_idx])

    # Format chromosome with leading zero if < 10
    # chr_fmt = f"{int(chr_name):02d}" if chr_name.isdigit() and int(chr_name) < 10 else chr_name
    chr_fmt = str(chr_name)
    chrom_found_list.append(chr_name)

    # Use dict keys for uniqueness, keeps insertion order
    chrom_ranges[chr_name][f"chr{chr_fmt}:{chr_pos}-{chr_pos}"] = None


100%|██████████| 1273891/1273891 [00:01<00:00, 651857.41it/s]


In [13]:
# Write unique SNPs per chromosome, split into multiple files if needed
for chr_name, snps_dict in tqdm(chrom_ranges.items()):
    snps_list = list(snps_dict.keys())
    total_snps = len(snps_list)
    print(f":) Please note that chromosome {chr_name} has {total_snps:,} total {genome_build} human variants or SNPs for score file {pgs_id}")
    if total_snps <= snp_threshold:
        # Single file
        chr_file_ranges = os.path.join(subdir1, f"{pgs_id}_chr{chr_name}_{genome_build}_ranges.txt")
        with open(chr_file_ranges, "w") as cf:
            for entry in snps_list:
                cf.write(entry + "\n")
    else:
        # Split into multiple files
        num_parts = math.ceil(total_snps / snp_threshold)
        for i in range(num_parts):
            start_idx = i * snp_threshold
            end_idx = min(start_idx + snp_threshold, total_snps)
            part_snps = snps_list[start_idx:end_idx]
            
            chr_file_ranges = os.path.join(
                subdir1, 
                f"{scores_list_names}_chr{chr_name}_{genome_build}_ranges_p{i+1}_of_{num_parts}_{hg38_genotype_imputation_type}_imputation.txt"
            )
            with open(chr_file_ranges, "w") as cf:
                for entry in part_snps:
                    cf.write(entry + "\n")

print(f"Created per-chromosome SNP coordinate files in {subdir}")


  9%|▉         | 2/22 [00:00<00:01, 13.27it/s]

:) Please note that chromosome 1 has 106,728 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 10 has 67,807 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 11 has 65,243 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 12 has 63,528 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 13 has 48,092 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 14 has 41,823 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 15 has 38,595 total GRCh38 human variants or SNPs for score file PGS005072


 77%|███████▋  | 17/22 [00:00<00:00, 52.98it/s]

:) Please note that chromosome 16 has 39,881 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 17 has 34,929 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 18 has 38,089 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 19 has 23,806 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 2 has 107,363 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 20 has 33,315 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 21 has 17,730 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 22 has 18,311 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 3 has 89,051 total GRCh38 human variants or SNPs for score file PGS005072
:) Please note that chromosome 4 has 79,840 tot

100%|██████████| 22/22 [00:00<00:00, 45.93it/s]

:) Please note that chromosome 9 has 58,077 total GRCh38 human variants or SNPs for score file PGS005072
Created per-chromosome SNP coordinate files in /Users/sk37/Desktop/hg38_demo/score_file/per_chrom_snp_files_GRCh38_combined_PGS005072


Example command in terminal:

dx upload /Users/sk37/Desktop/another_demo/score_file/per_chrom_snp_files_PGS005097/ranges_group/*.txt --destination IBD_Projects_Saniya:/second_saniya_demo/pgs_snps/ --brief

In [14]:
print(":) Please run the following command on your terminal after logging into DNAnexus to upload this score model file to ")
print(f" the following folder on DNAnexus portal for UK Biobank:  {dna_nexus_project_folder}pgs_scores/:\n")
command1 = f"dx upload {scores_fp + '*.txt'} --destination {dna_nexus_project_folder}pgs_scores/ --brief"
print(command1)

:) Please run the following command on your terminal after logging into DNAnexus to upload this score model file to 
 the following folder on DNAnexus portal for UK Biobank:  IBD_Projects_Saniya:/hg38_saniya_demo/pgs_scores/:

dx upload /Users/sk37/Desktop/hg38_demo/score_file/*.txt --destination IBD_Projects_Saniya:/hg38_saniya_demo/pgs_scores/ --brief


In [15]:
print(":) Please run the following command on your terminal after logging into DNAnexus to upload these score files per chromosome to ")
print(f" the following folder on DNAnexus portal for UK Biobank:  {dna_nexus_project_folder}pgs_snps/:\n")
command2 = f"dx upload {subdir1}/*.txt --destination {dna_nexus_project_folder}pgs_snps/ --brief"
print(command2)

:) Please run the following command on your terminal after logging into DNAnexus to upload these score files per chromosome to 
 the following folder on DNAnexus portal for UK Biobank:  IBD_Projects_Saniya:/hg38_saniya_demo/pgs_snps/:

dx upload /Users/sk37/Desktop/hg38_demo/score_file/per_chrom_snp_files_GRCh38_combined_PGS005072/ranges_group/*.txt --destination IBD_Projects_Saniya:/hg38_saniya_demo/pgs_snps/ --brief


# Creating the Samplesheet for Step 6 (in advance)

In [16]:
out_file = parent_file + "samplesheet/"
sampleset = "ukbiobank"
details = f"all_chromosomes_SNPs_merged_{scores_list_names}_{genome_build}_{hg38_genotype_imputation_type}"

outname = f"pgsc_calc_{scores_list_names}_{genome_build}_samplesheet_{hg38_genotype_imputation_type}.csv"
details

'all_chromosomes_SNPs_merged_PGS005072_GRCh38_topmed'

In [17]:
list_info = [sampleset, details, "", "bfile"]
list_df = pd.DataFrame(list_info).T
list_df.columns = ["sampleset", "path_prefix", "chrom", "format"]
list_df

,sampleset,path_prefix,chrom,format
0,ukbiobank,all_chromosomes_SNPs_merged_PGS005072_GRCh38_t...,,bfile


In [18]:
out_fp = out_file + outname
print(out_fp)
list_df.to_csv(out_fp, index = False)
list_df

/Users/sk37/Desktop/hg38_demo/samplesheet/pgsc_calc_PGS005072_GRCh38_samplesheet_topmed.csv


,sampleset,path_prefix,chrom,format
0,ukbiobank,all_chromosomes_SNPs_merged_PGS005072_GRCh38_t...,,bfile


In [19]:
print(":) Please run the following command on your terminal after logging into DNAnexus to upload the samplesheet for PGSC Calculator to ")
print(f" the following folder on DNAnexus portal for UK Biobank:  {dna_nexus_project_folder}:\n")
command3 = f"dx upload {out_fp} --destination {dna_nexus_project_folder} --brief"
print(command3)

:) Please run the following command on your terminal after logging into DNAnexus to upload the samplesheet for PGSC Calculator to 
 the following folder on DNAnexus portal for UK Biobank:  IBD_Projects_Saniya:/hg38_saniya_demo/:

dx upload /Users/sk37/Desktop/hg38_demo/samplesheet/pgsc_calc_PGS005072_GRCh38_samplesheet_topmed.csv --destination IBD_Projects_Saniya:/hg38_saniya_demo/ --brief
